In [1]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install Ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip uninstall scikit-learn

Found existing installation: scikit-learn 0.24.2
Uninstalling scikit-learn-0.24.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-0.24.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-f7e03b3e.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.24.2


In [5]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyaudioprocessing 1.1.13 requires scikit-learn==0.24.2; python_version <= "3.7", but you have scikit-learn 1.0.2 which is incompatible.


In [6]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install pyAudioProcessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import itertools
import os
import sys
import pywt
import librosa
import sklearn
import scipy
import scipy.io
import scipy.io.wavfile
from scipy import signal
from scipy.stats import kurtosis, skew, tstd
from scipy.signal import periodogram, welch, butter, lfilter, freqz
from os import path
from pydub import AudioSegment
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, Dropout, AlphaDropout, Activation, Flatten, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping,ReduceLROnPlateau,ProgbarLogger
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, recall_score, classification_report, auc, roc_curve, precision_recall_fscore_support, f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from pyAudioProcessing.extract_features import get_features

In [9]:
#List of Labels - 0 for normal, 1 for murmur
labels = [0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,1]
len(labels)

23

In [10]:
#They were originally named differently and I have renamed and numbered them for ease of access.
#The michigan heart sound recordings are in mp3 format
#The recordings are converted to wav format for easy processing as most python libraries require the audio format to be wav.

def converttowav(filepath):
  destinationpath = filepath.split('.')[0] + '.wav'
  sound = AudioSegment.from_mp3(filepath)
  sound.export(destinationpath, format = "wav")

filepathslist = list()
for i in range(23):
  filepath = str(i+1) + '.mp3'
  converttowav(filepath)
  filepathslist.append(str(i+1)+'.wav')

In [11]:
#Creating an empty dataframe with columns to store filenames and the respective labels
dataframe = pd.DataFrame(columns = ['filename', 'labels'])

In [12]:
filepathslist

['1.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 '5.wav',
 '6.wav',
 '7.wav',
 '8.wav',
 '9.wav',
 '10.wav',
 '11.wav',
 '12.wav',
 '13.wav',
 '14.wav',
 '15.wav',
 '16.wav',
 '17.wav',
 '18.wav',
 '19.wav',
 '20.wav',
 '21.wav',
 '22.wav',
 '23.wav']

In [13]:
labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [14]:
labels[18]

0

In [15]:
#Inserting filename and label values into dataframe
dataframe['filename'] = filepathslist
dataframe['labels'] = labels

In [16]:
dataframenew = pd.DataFrame(columns = ['new ID', 'wav filename', 'labels'])

In [17]:
import random

In [18]:
newID = list()
wavfilename = list()
newlabels = list()
for i in range(3000):
  x = random.randint(1,23)
  newID.append(str(i+1))
  wavfilename.append(str(x)+'.wav')
  newlabels.append(labels[x-1])

In [19]:
dataframenew['new ID'] = newID
dataframenew['wav filename'] = wavfilename
dataframenew['labels'] = newlabels
dataframenew

,new ID,wav filename,labels
0,1,18.wav,0
1,2,16.wav,1
2,3,8.wav,1
3,4,1.wav,0
4,5,6.wav,1
...,...,...,...
2995,2996,6.wav,1
2996,2997,15.wav,1
2997,2998,12.wav,1
2998,2999,12.wav,1


In [20]:
dataframe

,filename,labels
0,1.wav,0
1,2.wav,0
2,3.wav,0
3,4.wav,0
4,5.wav,0
5,6.wav,1
6,7.wav,1
7,8.wav,1
8,9.wav,1
9,10.wav,1


In [21]:
dataframenew['labels'].value_counts()

1    1828
0    1172
Name: labels, dtype: int64

In [22]:
X_train, X_test, y_Train, y_Test = train_test_split(dataframenew['new ID'], dataframenew['labels'], train_size=0.8, shuffle=True)

In [23]:
X_train

1531    1532
1593    1594
1260    1261
2835    2836
142      143
        ... 
199      200
1091    1092
113      114
864      865
1555    1556
Name: new ID, Length: 2400, dtype: object

In [24]:
X_test

25        26
622      623
579      580
2965    2966
523      524
        ... 
101      102
2318    2319
1068    1069
1794    1795
2285    2286
Name: new ID, Length: 600, dtype: object

In [25]:
#Computing Features - 

#Discrete Wavelet Transform
def computeDWTs(x):
    #approximation and detail coefficients
    res = np.array([])
    ca, cd = pywt.dwt(x, 'db8')
    #Compute mean, stdev, skew, kurtosis
    camean = np.mean(ca)
    res = np.hstack((res,camean))
    castdev = tstd(ca)
    res = np.hstack((res,castdev))
    caskew = skew(ca)
    res = np.hstack((res,caskew))
    cakurtosis = kurtosis(ca)
    res = np.hstack((res,cakurtosis))
    cdmean = np.mean(cd)
    res = np.hstack((res,cdmean))
    cdstdev = tstd(cd)
    res = np.hstack((res,cdstdev))
    cdskew = skew(cd)
    res = np.hstack((res,cdskew))
    cdkurtosis = kurtosis(cd)
    res = np.hstack((res, cdkurtosis))
    return res

#Computing the Spectral Entropy of the signal - Shannon Entropy of the Power Spectral Density (PSD) of the data
def SpectralEntropy(Signal, SampleRate, method = 'fft', nperseg = None, normalize = False):
    Signal = np.array(Signal)
    if method == 'fft':
      _, psd = periodogram(Signal, SampleRate)
    elif method == 'welch':
      _, psd = welch(Signal, SampleRate, nperseg=nperseg)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        se /= np.log2(psd_norm.size)
    return se

#Computing the Power Spectral Density(PSD) of signal - 
def computepsd(x):
    freq, psd = signal.welch(x)
    return psd

#Computing Mel-frequency cepstral coefficients, Gammatone frequency cepstral coefficients, RMS Energy, Zero-crossing rate, mean, variance, skew, and kurtosis of the signal
def computeMFCC(x, sr):
    mfccs = np.mean(librosa.feature.mfcc(y = x, sr = sr, n_mfcc = 27).T, axis = 0)
    return mfccs

def computeGFCC(file_name):
    gfcc = get_features(file=file_name, feature_names=["gfcc"])
    gfcc = np.array(gfcc['audio'][file_name]['features'])
    return gfcc

def computeRMSEn(x):
    rmsen = np.mean(librosa.feature.rms(x, frame_length=2048, hop_length=1024, center=True).T, axis = 0)
    return rmsen

def computeZCR(x):
    zcr = np.mean(librosa.feature.zero_crossing_rate(y = x, frame_length=2048, hop_length=1024).T, axis = 0)
    return zcr

def computestats(x):
#Mean, Variance, Skew, Kurtosis
    stats = []
    me = np.mean(x)
    stats.append(me)
    variance = np.var(x)
    stats.append(variance)
    sk = skew(x)
    stats.append(sk)
    kurt = kurtosis(x)
    stats.append(kurt)
    return stats

In [26]:
#Function to extract the required features
def extractfeatures(file_name, x, sr, *, mfcc: bool = False, dwt: bool = False, spectral_entropy: bool = False, psd: bool = False, gfcc: bool = False, rmse: bool = False, zcr: bool = False, stats: bool = False):
    finalfeatures = np.array([])
    if mfcc:
        mfccs = computeMFCC(x, sr)
        finalfeatures = np.hstack((finalfeatures, mfccs))
    if dwt:
        dwtr = computeDWTs(x)
        finalfeatures = np.hstack((finalfeatures, dwtr))
    if spectral_entropy:
        specen = SpectralEntropy(x, sr)
        finalfeatures = np.hstack((finalfeatures,specen))
    if psd:
        psds = computepsd(x)
        finalfeatures = np.hstack((finalfeatures, psds))
    if gfcc:
        gfccs = computeGFCC(file_name)
        finalfeatures = np.hstack((finalfeatures, gfccs))
    if rmse:
        rmses = computeRMSEn(x)
        finalfeatures = np.hstack((finalfeatures, rmses))
    if zcr:
        zcrs = computeZCR(x)
        finalfeatures = np.hstack((finalfeatures, zcrs))
    if stats:
        stats1 = computestats(x)
        finalfeatures = np.hstack((finalfeatures, stats1))
    return finalfeatures

In [27]:
#loads audio and extracts required features
def load_file_data(file_num, duration=10, sr = 4000):
    input_length=sr*duration
    notfeatures = []
    i = 0
    for file_no in file_num:
        try:
            #sound_file = folder + '/' + file_name
            sfile = int(file_no)
            sound_file = dataframenew['wav filename'][sfile-1]
            print ("load file ", i, sound_file)
            x, sr = librosa.load(sound_file,duration=duration,res_type='kaiser_fast')
            dur = librosa.get_duration(y=x, sr=sr)
            #Padding audio signal
            if (round(dur) < duration):
                print ("fixing audio length:", file_no, sound_file)
                y = librosa.util.fix_length(x, input_length)
            feature = extractfeatures(sound_file, x, sr, mfcc = True, dwt = True, psd = True, rmse = True, zcr = True,stats = True)
            notfeatures.append(feature)          
        except Exception as e:
            print(e)
            #notfeatures = np.asarray(features, dtype=np.float32)
        i = i+1 
    return notfeatures

In [28]:
x_train = []
x_test = []
x_train = load_file_data(X_train)
x_test = load_file_data(X_test)

load file  0 3.wav
load file  1 17.wav
load file  2 4.wav
load file  3 14.wav
load file  4 6.wav
load file  5 2.wav
load file  6 4.wav
load file  7 12.wav
load file  8 6.wav
load file  9 20.wav
load file  10 13.wav
load file  11 3.wav
load file  12 20.wav
load file  13 6.wav
load file  14 21.wav
load file  15 1.wav
load file  16 19.wav
load file  17 9.wav
load file  18 23.wav
load file  19 21.wav
load file  20 2.wav
load file  21 9.wav
load file  22 19.wav
load file  23 6.wav
load file  24 21.wav
load file  25 12.wav
load file  26 12.wav
load file  27 10.wav
load file  28 8.wav
load file  29 16.wav
load file  30 16.wav
load file  31 12.wav
load file  32 3.wav
load file  33 7.wav
load file  34 1.wav
load file  35 9.wav
load file  36 19.wav
load file  37 22.wav
load file  38 12.wav
load file  39 17.wav
load file  40 2.wav
load file  41 15.wav
load file  42 21.wav
load file  43 6.wav
load file  44 15.wav
load file  45 14.wav
load file  46 20.wav
load file  47 4.wav
load file  48 23.wav
lo

In [29]:
x_train = np.array(x_train)
x_test = np.asarray(x_test)

In [30]:
x_test.shape

(600, 170)

In [31]:
x_train.dtype

dtype('float64')

In [32]:
y_Train = np.asarray(y_Train)

In [33]:
y_Test = np.asarray(y_Test)

In [34]:
traindataframe = pd.DataFrame(columns = ['x_train ID', 'labels'])
testdataframe = pd.DataFrame(columns = ['x_test ID', 'labels'])
traindataframe['x_train ID'] = X_train
testdataframe['x_test ID'] = X_test
traindataframe['labels'] = y_Train
testdataframe['labels'] = y_Test

In [35]:
testdataframe

,x_test ID,labels
25,26,0
622,623,1
579,580,1
2965,2966,0
523,524,1
...,...,...
101,102,0
2318,2319,1
1068,1069,1
1794,1795,1


In [36]:
xtrainwavfiles = list()
for i in X_test:
  num = int(i)
  xtrainwavfiles.append(dataframenew['wav filename'][num-1])

In [37]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import kerastuner
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
def model_builder(hp):
  model = Sequential()
  hp_units1 = hp.Int('units_', min_value=2, max_value=64, step=4)
  model.add(tf.keras.layers.Dense(units = hp_units1, input_dim = x_train.shape[1]))

  # Tune the number of dense layers
  for i in range(hp.Int('num_layers', 1, 5)):
    
    # Tune the number of units in the each dense layer
    hp_units = hp.Int('units_'+str(i), min_value=2, max_value=64, step=4)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    
    # Tune the dropout rate in the each dense layer
    hp_dropout = hp.Float('rate', min_value=0.0, max_value=0.5, step=0.1)
    model.add(tf.keras.layers.Dropout(hp_dropout))
    
  # Add dense output layer
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  # Tune the learning rate for the optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy',
                metrics=[tf.keras.metrics.Recall()])

  return model

#tuner = BayesianOptimization(model_builder,
#max_trials=50,
#objective='val_accuracy',
#overwrite=True)

tuner = Hyperband(model_builder,
objective=kerastuner.Objective("val_recall", direction="max"),
max_epochs=50,
factor=3, overwrite = True)

tuner.search(x_train,y_Train, epochs=50, validation_split=0.2)
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)

Trial 90 Complete [00h 00m 14s]
val_recall: 1.0

Best val_recall So Far: 1.0
Total elapsed time: 00h 06m 35s


In [39]:
history = model.fit(x_train, y_Train, batch_size = 10, epochs=50, validation_split = 0.2, verbose = 1)

Epoch 1/50
192/192 [==============================] - 2s 6ms/step - loss: 1.6935 - recall_1: 0.8238 - val_loss: 0.6234 - val_recall_1: 1.0000
Epoch 2/50
192/192 [==============================] - 1s 3ms/step - loss: 0.6206 - recall_1: 0.9949 - val_loss: 0.4932 - val_recall_1: 1.0000
Epoch 3/50
192/192 [==============================] - 1s 3ms/step - loss: 0.4006 - recall_1: 0.9574 - val_loss: 0.2772 - val_recall_1: 1.0000
Epoch 4/50
192/192 [==============================] - 1s 3ms/step - loss: 0.3079 - recall_1: 0.9864 - val_loss: 0.2083 - val_recall_1: 1.0000
Epoch 5/50
192/192 [==============================] - 1s 3ms/step - loss: 0.2576 - recall_1: 0.9949 - val_loss: 0.1732 - val_recall_1: 1.0000
Epoch 6/50
192/192 [==============================] - 1s 3ms/step - loss: 0.2395 - recall_1: 0.9957 - val_loss: 0.1737 - val_recall_1: 1.0000
Epoch 7/50
192/192 [==============================] - 1s 3ms/step - loss: 0.2664 - recall_1: 0.9966 - val_loss: 0.2039 - val_recall_1: 1.0000
Epoch 

In [40]:
y_predict = np.argmax(model.predict(x_test), axis=-1)
y_pred = model.predict(x_test)
y_pred=np.argmax(y_pred, axis=1)

19/19 [==============================] - 0s 2ms/step


In [41]:
print("Classification Report : \n\n", classification_report(y_Test, y_pred))

Classification Report : 

               precision    recall  f1-score   support

           0       0.40      1.00      0.57       237
           1       0.00      0.00      0.00       363

    accuracy                           0.40       600
   macro avg       0.20      0.50      0.28       600
weighted avg       0.16      0.40      0.22       600



In [42]:
df = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred})
df

,Input,Wav filename,Real Values,Predicted Values
25,26,1.wav,0,0
622,623,12.wav,1,0
579,580,22.wav,1,0
2965,2966,4.wav,0,0
523,524,12.wav,1,0
...,...,...,...,...
101,102,4.wav,0,0
2318,2319,10.wav,1,0
1068,1069,11.wav,1,0
1794,1795,8.wav,1,0


In [43]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(x_train, y_Train)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [44]:
clf.score(x_test, y_Test)

1.0

In [45]:
y_pred1 = clf.predict(x_test)
print("Classification Report : \n\n", classification_report(y_Test, y_pred1))

Classification Report : 

               precision    recall  f1-score   support

           0       1.00      1.00      1.00       237
           1       1.00      1.00      1.00       363

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



In [46]:
df1 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred1})
df1

,Input,Wav filename,Real Values,Predicted Values
25,26,1.wav,0,0
622,623,12.wav,1,1
579,580,22.wav,1,1
2965,2966,4.wav,0,0
523,524,12.wav,1,1
...,...,...,...,...
101,102,4.wav,0,0
2318,2319,10.wav,1,1
1068,1069,11.wav,1,1
1794,1795,8.wav,1,1


In [47]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [48]:
clf2 = LinearSVC(penalty='l2', loss='hinge')
clf2.fit(x_train, y_Train)
clf2.score(x_test, y_Test)

1.0

In [49]:
y_pred2 = clf2.predict(x_test)
print("Classification Report : \n\n", classification_report(y_Test, y_pred2))

Classification Report : 

               precision    recall  f1-score   support

           0       1.00      1.00      1.00       237
           1       1.00      1.00      1.00       363

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



In [50]:
df2 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred2})
df2

,Input,Wav filename,Real Values,Predicted Values
25,26,1.wav,0,0
622,623,12.wav,1,1
579,580,22.wav,1,1
2965,2966,4.wav,0,0
523,524,12.wav,1,1
...,...,...,...,...
101,102,4.wav,0,0
2318,2319,10.wav,1,1
1068,1069,11.wav,1,1
1794,1795,8.wav,1,1


In [51]:
clf3 = SVC(kernel = 'rbf', random_state = 0)
clf3.fit(x_train, y_Train)

SVC(random_state=0)

In [52]:
clf3.score(x_test, y_Test)

0.675

In [53]:
y_pred3 = clf3.predict(x_test)
print("Classification Report : \n\n", classification_report(y_Test, y_pred3))

Classification Report : 

               precision    recall  f1-score   support

           0       1.00      0.18      0.30       237
           1       0.65      1.00      0.79       363

    accuracy                           0.68       600
   macro avg       0.83      0.59      0.54       600
weighted avg       0.79      0.68      0.60       600



In [54]:
df3 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred3})
df3

,Input,Wav filename,Real Values,Predicted Values
25,26,1.wav,0,0
622,623,12.wav,1,1
579,580,22.wav,1,1
2965,2966,4.wav,0,0
523,524,12.wav,1,1
...,...,...,...,...
101,102,4.wav,0,0
2318,2319,10.wav,1,1
1068,1069,11.wav,1,1
1794,1795,8.wav,1,1


In [55]:
clf4 = SGDClassifier(loss = 'hinge').fit(x_train, y_Train)

In [56]:
clf4.score(x_test, y_Test)

0.955

In [57]:
y_pred4 = clf4.predict(x_test)
#y_pred=np.argmax(y_pred, axis=1)
print("Classification Report : \n\n", classification_report(y_Test, y_pred4))

Classification Report : 

               precision    recall  f1-score   support

           0       1.00      0.89      0.94       237
           1       0.93      1.00      0.96       363

    accuracy                           0.95       600
   macro avg       0.97      0.94      0.95       600
weighted avg       0.96      0.95      0.95       600



In [58]:
df4 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred4})
df4

,Input,Wav filename,Real Values,Predicted Values
25,26,1.wav,0,0
622,623,12.wav,1,1
579,580,22.wav,1,1
2965,2966,4.wav,0,0
523,524,12.wav,1,1
...,...,...,...,...
101,102,4.wav,0,0
2318,2319,10.wav,1,1
1068,1069,11.wav,1,1
1794,1795,8.wav,1,1
